<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/CO2_Buildup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# CO2 Input-only Atmosphere–Ocean Mini-Model (Colab, no-weathering)
# Empty start; constant volcanic D. Dropdowns for f_atm_eq, tau_ex, single ECS.
# ΔT is the equilibrium temperature anomaly relative to 280 ppm (preindustrial):
#   ΔT = ECS * log2( A_ppm / 280 )
# Outputs: analytic/sim time-to-target, partition (A ppm, O GtCO2, O ppm-eq),
#          Plot 1: A(t) vs time; Plot 2: cumulative partition A (ppm) & O/7.8 (ppm-eq) vs time.
# &Scenarios
#
# A. Low f_atm_eq, Low tau_ex
# Atmosphere is very small and exchange is fast.
# Volcanic CO₂ is quickly absorbed into the ocean.
#
# B. High f_atm_eq, Low tau_ex
# Atmosphere is large in equilibrium and exchange is fast.
# Atmospheric CO₂ rises sharply.
#
# C. Low f_atm_eq, High tau_ex
# Atmosphere should be small in equilibrium, but exchange is extremely slow.
# CO₂ accumulates in the atmosphere for millennia before oceans absorb it.
#
# D. High f_atm_eq, High tau_ex
# Atmosphere is large and oceans absorb slowly.
# Persistent high atmospheric CO₂ levels.

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Conversions
PPM_TO_GtCO2 = 7.8
PPM_TO_MtCO2 = 7800.0
GtCO2_TO_PPM = 1.0 / PPM_TO_GtCO2

PREINDUSTRIAL_CO2 = 280.0  # ppm (fixed reference for ΔT)

# --- Core simulation: input-only (no sinks), two-box (A in ppm, O in GtCO2)
def simulate_constant_D(f_atm_eq, tau_ex, D_Mt=300.0, target_ppm=560.0, dt=1.0):
    f_atm_eq = max(1e-6, float(f_atm_eq))
    tau_ex   = max(1e-6, float(tau_ex))
    D_Mt     = max(0.0,  float(D_Mt))
    target_ppm = max(0.0, float(target_ppm))
    dt = max(1e-3, float(dt))

    D_ppm = D_Mt / PPM_TO_MtCO2
    required_total = target_ppm / f_atm_eq                 # ppm-equivalent total needed
    t_est = required_total / (D_ppm if D_ppm > 0 else 1e-12)

    years = int(math.ceil(t_est * 1.5 + 1000))
    years = max(years, 2000)
    n = int(years/dt) + 1

    t = np.arange(n, dtype=float) * dt
    A = np.zeros(n)  # ppm
    O = np.zeros(n)  # GtCO2

    hit_index = None
    for i in range(1, n):
        C_tot_ppm = A[i-1] + O[i-1] * GtCO2_TO_PPM
        A_eq = f_atm_eq * C_tot_ppm
        F_ex_ppm = (A[i-1] - A_eq) / tau_ex  # + = atmosphere -> ocean

        A[i] = A[i-1] + (D_ppm - F_ex_ppm) * dt
        O[i] = O[i-1] + (F_ex_ppm * PPM_TO_GtCO2) * dt

        if hit_index is None and A[i] >= target_ppm:
            hit_index = i
            break

    if hit_index is not None:
        i_end = hit_index
        t_hit = t[hit_index]
    else:
        i_end = n - 1
        t_hit = None

    A_end_ppm = float(A[i_end])
    O_end_Gt = float(O[i_end])
    O_end_ppm_eq = O_end_Gt * GtCO2_TO_PPM

    return {
        "t": t, "A": A, "O": O,
        "t_est": float(t_est), "t_hit": t_hit,
        "A_end_ppm": A_end_ppm,
        "O_end_Gt": O_end_Gt,
        "O_end_ppm_eq": O_end_ppm_eq
    }

# --- Dropdown scenarios (f, tau, ECS)
tau_options = [50, 500, 2000, 5000]           # years
f_options   = [0.020, 0.030, 0.050, 0.070]    # fractions
ECS_choices = [2.0, 2.5, 3.0, 3.5, 4.0, 4.5]  # °C per doubling

tau_dropdown = widgets.Dropdown(options=tau_options, value=500, description="tau_ex (yr)")
f_dropdown   = widgets.Dropdown(options=f_options, value=0.030, description="f_atm_eq")
ECS_dropdown = widgets.Dropdown(options=ECS_choices, value=3.0, description="ECS (°C/2x)")

# --- Numerical inputs
D_box      = widgets.FloatText(value=300.0, description="D (Mt/yr)", step=10.0)
target_box = widgets.FloatText(value=560.0, description="target (ppm)", step=10.0)
dt_box     = widgets.FloatText(value=1.0, description="dt (yr)", step=0.5)

# --- Reset & Output
reset_button = widgets.Button(description="Reset to defaults", button_style="warning")
out = widgets.Output()

# --- Helpers
def fmt_years(val):
    if val is None:
        return "not reached"
    return f"{val/1000:,.0f} kyr" if val >= 10000 else f"{val:,.0f} yr"

def deltaT_from_co2(A_ppm, ECS):
    # ΔT (°C) anomaly vs 280 ppm (preindustrial).
    if ECS is None or ECS <= 0 or A_ppm <= 0:
        return np.nan
    return ECS * (math.log(A_ppm / PREINDUSTRIAL_CO2, 2.0))

def run_model(change=None):
    with out:
        clear_output(wait=True)

        f_val   = f_dropdown.value
        tau_val = tau_dropdown.value
        D_val   = D_box.value
        target  = target_box.value
        dt      = dt_box.value
        ECS     = ECS_dropdown.value

        res = simulate_constant_D(f_atm_eq=f_val, tau_ex=tau_val, D_Mt=D_val,
                                  target_ppm=target, dt=dt)

        dT = deltaT_from_co2(res["A_end_ppm"], ECS)

        # Table
        df = pd.DataFrame([{
            "f_atm_eq": f_val,
            "tau_ex (yr)": tau_val,
            "D (Mt/yr)": D_val,
            "target (ppm)": target,
            "analytic_time_est": fmt_years(res["t_est"]),
            "sim_time_to_target": fmt_years(res["t_hit"]),
            "Atmosphere_end (ppm)": res["A_end_ppm"],
            "Ocean_end (GtCO2)": res["O_end_Gt"],
            "Ocean_end (ppm-eq)": res["O_end_ppm_eq"],
            "ECS (°C/2x)": ECS,
            "ΔT vs 280 ppm (°C)": dT
        }])

        display(df.style.format({
            "Atmosphere_end (ppm)": "{:,.2f}",
            "Ocean_end (GtCO2)": "{:,.2f}",
            "Ocean_end (ppm-eq)": "{:,.2f}",
            "ΔT vs 280 ppm (°C)": "{:,.2f}",
        }))

        # Plot 1: Atmospheric CO2 vs time
        t, A, O = res["t"], res["A"], res["O"]
        plt.figure(figsize=(8,5))
        plt.plot(t, A, label=f"A (ppm): f={f_val}, tau={int(tau_val)} yr")
        plt.axhline(target, linestyle="--")
        if res["t_hit"] is not None:
            plt.axvline(res["t_hit"], linestyle="--")
        plt.xlabel("Time (years)")
        plt.ylabel("Atmospheric CO$_2$ (ppm)")
        plt.title("Atmospheric CO$_2$ vs time (constant D, empty start; no weathering)")
        plt.legend()
        plt.grid(True, linestyle=":")
        plt.show()

        # Plot 2 (replacement): cumulative partition in ppm-equivalents
        O_ppm_eq = O * GtCO2_TO_PPM
        plt.figure(figsize=(8,5))
        plt.plot(t, A, label="Atmosphere (ppm)")
        plt.plot(t, O_ppm_eq, label="Ocean (ppm-eq)")
        plt.xlabel("Time (years)")
        plt.ylabel("CO$_2$ (ppm-equivalent)")
        plt.title("Cumulative partition: atmosphere vs ocean (ppm & ppm-eq)")
        plt.legend()
        plt.grid(True, linestyle=":")
        plt.show()

        print("NOTE: ΔT is an equilibrium anomaly relative to 280 ppm (preindustrial).")
        print("      This input-only model excludes weathering and slow feedbacks; CO₂ accumulates indefinitely.")

def reset_fields(_):
    f_dropdown.value   = 0.030
    tau_dropdown.value = 500
    D_box.value        = 300.0
    target_box.value   = 560.0
    dt_box.value       = 1.0
    ECS_dropdown.value = 3.0

# Wire up
for w in [tau_dropdown, f_dropdown, D_box, target_box, dt_box, ECS_dropdown]:
    w.observe(run_model, names="value")
reset_button.on_click(reset_fields)

# Layout
ui = widgets.VBox([
    widgets.HBox([D_box, target_box, dt_box]),
    widgets.HBox([f_dropdown, tau_dropdown]),
    widgets.HBox([ECS_dropdown]),
    reset_button,
    out
])

display(ui)
run_model()